In [54]:
import sys
sys.path.append("./")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")

from typing import TYPE_CHECKING
if TYPE_CHECKING:
    from snakemake_stub import *
from Bio import SeqIO
import scipy.sparse as sp
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors  
import numpy as np
from sim_meta_function import load_reads,build_kmer_index,build_feature_matrix,get_table,get_simhash,evaluate

def init_reverse_complement():
    TRANSLATION_TABLE = str.maketrans("ACTGactg", "TGACtgac")

    def reverse_complement(sequence: str) -> str:
        """
        >>> reverse_complement("AATC")
        'GATT'
        >>> reverse_complement("CCANT")
        'ANTGG'
        """
        sequence = str(sequence)
        return sequence.translate(TRANSLATION_TABLE)[::-1]

    return reverse_complement
reverse_complement = init_reverse_complement()

ref_database = '/home/miaocj/docker_dir/data/metagenome/bacteria/part1.fa'
query_reads = '/home/miaocj/docker_dir/data/metagenome/bacteria/pbsim_ONT_98_30k_10dep_part1_reads.fasta'

k=30000
split_ref = []
ref_reads_tax_list = []
with open(ref_database, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        tax=record.description.split(' ')[1]+record.description.split(' ')[2]
        k = int(len(record.seq)/2)
        for p in range(0,len(record.seq) - k + 1,int(len(record.seq)/4)):
            kmer = record.seq[p : p + k]
            split_ref.append(kmer)
            ref_reads_tax_list.append(tax)
            split_ref.append(reverse_complement(kmer))
split_ref_tax_dict = {i:tax for i,tax in enumerate(ref_reads_tax_list)}

ref_reads_tax_list = []
with open(ref_database) as file:
    for lines in file:
        if lines[0] == '>':
            line = lines.strip().split(' ')
            ref_reads_tax_list.append(line[1]+line[2])
ref_read_tax = {i:tax for i,tax in enumerate(ref_reads_tax_list)}

flag = 0
que_read_tax = {}
with open(query_reads) as file:
    for lines in file:
        if lines[0] == '>':
            start = lines.index('S')
            end = lines.index('_')
            ref_num = lines[start+1:end]
            que_read_tax[flag] = ref_read_tax[int(ref_num)-1]
            flag +=1
print("loading reads")
print("loading done\nbuilding kmer index")
sample_fraction=0.1
min_multiplicity=2
seed=562104830
kmer_indices = build_kmer_index(        
        read_sequences=split_ref,
        k=16,
        sample_fraction=sample_fraction,
        min_multiplicity=min_multiplicity,
        seed=seed)
print("done\nloading query reads")
qread_names, qread_orientations, qread_sequences = load_reads(query_reads)
print("done\nbuilding feature matrix")

ref_feature_matrix,ref_read_features = build_feature_matrix(read_sequences=split_ref,kmer_indices=kmer_indices,k=16)
que_feature_matrix,que_read_features = build_feature_matrix(read_sequences=qread_sequences,kmer_indices=kmer_indices,k=16)
kmer_num = que_feature_matrix.shape[1]
print("done\nbuilding hash table")
hash_table = get_table(kmer_num, seed = 4829) 
print("done\nget ref simhash")
ref_reads_simhash_array = get_simhash(ref_read_features,hash_table)
print("ref done\nget que simhash")
que_reads_simhash_array = get_simhash(que_read_features,hash_table) 

print("que done\nget neignbors")
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric=hamming_distance)
nbrs.fit(ref_reads_simhash_array)
indices = nbrs.kneighbors(que_reads_simhash_array,return_distance=False)
print("done\nevaluates")

##evaluate
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices,split_ref_tax_dict,que_read_tax)
print(precision,sensitivity,precision_sep,sensitivity_sep)

done
building hash table
done
get ref simhash
ref done
get que simhash
que done
get neignbors
done
evaluates
0.7152036832588234 0.694840426226143 [0.78196286 0.78135676 0.64336158 0.68130181 0.48419637 0.71889671
 0.82108358 0.8583196  0.69861627 0.66733295 0.73081201] [0.5891287  0.59296296 0.85781544 0.72658808 0.77586207 0.6400209
 0.68789585 0.64480198 0.646875   0.7504817  0.73081201]


In [63]:
hash_table.shape

(8528703, 3200)

In [75]:
import mmh3
repeat=60
kmer_num = 15055916
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=bool) 
    return hash_array  
hash_table = np.empty((kmer_num,repeat,32),dtype=bool) 
rng = np.random.default_rng(4829)  
hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)   
for flag,seed in enumerate(hash_seeds):
    print(flag)
    for kmer_index in range(kmer_num):
        hash_table[kmer_index,flag,:]=_hash(kmer_index, seed=seed)
        new_hash_table=np.reshape(hash_table,(kmer_num,32*repeat))

0
1
2
3
4
5
6
7
8


KeyboardInterrupt: 

In [13]:
hash_table = get_table(kmer_num, seed = 4829) 
print("done\nget ref simhash")
ref_reads_simhash_array = get_simhash(ref_read_features,hash_table)
print("ref done\nget que simhash")
que_reads_simhash_array = get_simhash(que_read_features,hash_table) 

print("que done\nget neignbors")
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric=hamming_distance)
nbrs.fit(ref_reads_simhash_array)
indices = nbrs.kneighbors(que_reads_simhash_array,return_distance=False)
print("done\nevaluates")

##evaluate
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices,split_ref_tax_dict,que_read_tax)
print(precision,sensitivity,precision_sep,sensitivity_sep)

done
get ref simhash
ref done
get que simhash
que done
get neignbors
done
evaluates


/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.058475234168521205 0.07869494294446369 [0.12976023 0.04439615 0.03013315 0.27202826 0.         0.
 0.         0.05125285 0.04351965 0.02596315 0.04617414] [0.11031175 0.05296296 0.04048964 0.42861821 0.         0.
 0.         0.04455446 0.1        0.04977521 0.03893215]


/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [39]:
from sklearn.metrics import precision_score, recall_score  
actual = []
prediction = []
for query_read_num,x in enumerate(indices):
    neighbor = x[0]
    query_read_num = (query_read_num-1)/2  if query_read_num %2 !=0 else query_read_num/2
    prediction.append(split_ref_tax_dict[neighbor])
    actual.append(que_read_tax[int(query_read_num)])

precision = precision_score(actual, prediction,average='macro')
sensitivity = recall_score(actual, prediction,average='macro')
##计算每个类别的
precision_sep = precision_score(actual, prediction, average=None)  
sensitivity_sep = recall_score(actual, prediction, average=None)
print(precision,sensitivity,precision_sep,sensitivity_sep)


In [40]:
print(precision,sensitivity,precision_sep,sensitivity_sep)

0.5754829836662846 0.5228086497804562 [0.63057671 0.63864565 0.4498503  0.56190041 0.31130141 0.5735568
 0.69009166 0.74151735 0.52105443 0.58627563 0.62554247] [0.46322942 0.40518519 0.56591337 0.40275049 0.6953125  0.48275862
 0.55629838 0.48143564 0.475625   0.66120745 0.56117909]


In [32]:
ref_reads_tax_list = []
with open(ref_database) as file:
    for lines in file:
        if lines[0] == '>':
            line = lines.strip().split(' ')
            ref_reads_tax_list.append(line[1]+line[2])
ref_read_tax = {i:tax for i,tax in enumerate(ref_reads_tax_list)}

flag = 0
que_read_tax = {}
with open(query_reads) as file:
    for lines in file:
        if lines[0] == '>':
            start = lines.index('S')
            end = lines.index('_')
            ref_num = lines[start+1:end]
            que_read_tax[flag] = ref_read_tax[int(ref_num)-1]
            flag +=1

In [34]:
que_read_tax

{0: 'Escherichiacoli',
 1: 'Escherichiacoli',
 2: 'Escherichiacoli',
 3: 'Escherichiacoli',
 4: 'Escherichiacoli',
 5: 'Escherichiacoli',
 6: 'Escherichiacoli',
 7: 'Escherichiacoli',
 8: 'Escherichiacoli',
 9: 'Escherichiacoli',
 10: 'Escherichiacoli',
 11: 'Escherichiacoli',
 12: 'Escherichiacoli',
 13: 'Escherichiacoli',
 14: 'Escherichiacoli',
 15: 'Escherichiacoli',
 16: 'Escherichiacoli',
 17: 'Escherichiacoli',
 18: 'Escherichiacoli',
 19: 'Escherichiacoli',
 20: 'Escherichiacoli',
 21: 'Escherichiacoli',
 22: 'Escherichiacoli',
 23: 'Escherichiacoli',
 24: 'Escherichiacoli',
 25: 'Escherichiacoli',
 26: 'Escherichiacoli',
 27: 'Escherichiacoli',
 28: 'Escherichiacoli',
 29: 'Escherichiacoli',
 30: 'Escherichiacoli',
 31: 'Escherichiacoli',
 32: 'Escherichiacoli',
 33: 'Escherichiacoli',
 34: 'Escherichiacoli',
 35: 'Escherichiacoli',
 36: 'Escherichiacoli',
 37: 'Escherichiacoli',
 38: 'Escherichiacoli',
 39: 'Escherichiacoli',
 40: 'Escherichiacoli',
 41: 'Escherichiacoli',
 4

In [44]:
np.savez('/home/miaocj/docker_dir/data/metagenome/hash_table.npz',hash_table)

In [50]:
hash_table

array([[-1, -1,  1, ...,  1,  1, -1],
       [-1,  1, -1, ..., -1,  1,  1],
       [-1,  1, -1, ..., -1, -1,  1],
       ...,
       [-1, -1,  1, ..., -1,  1, -1],
       [-1,  1, -1, ...,  1, -1,  1],
       [ 1,  1,  1, ...,  1, -1, -1]], dtype=int8)

In [45]:
new = np.load('/home/miaocj/docker_dir/data/metagenome/hash_table.npz')

In [49]:
new['arr_0']

array([[-1, -1,  1, ...,  1,  1, -1],
       [-1,  1, -1, ..., -1,  1,  1],
       [-1,  1, -1, ..., -1, -1,  1],
       ...,
       [-1, -1,  1, ..., -1,  1, -1],
       [-1,  1, -1, ...,  1, -1,  1],
       [ 1,  1,  1, ...,  1, -1, -1]], dtype=int8)

In [5]:
ref_reads_tax_list = []
with open(ref_database) as file:
    for lines in file:
        if lines[0] == '>':
            line = lines.strip().split(' ')
            ref_reads_tax_list.append(line[1]+line[2])
ref_read_tax = {i:tax for i,tax in enumerate(ref_reads_tax_list)}

flag = 0
que_read_tax = {}
with open(query_reads) as file:
    for lines in file:
        if lines[0] == '>':
            start = lines.index('S')
            end = lines.index('_')
            ref_num = lines[start+1:end]
            que_read_tax[flag] = ref_read_tax[int(ref_num)-1]
            flag +=1
que_read_tax

{0: 'Escherichiacoli',
 1: 'Escherichiacoli',
 2: 'Escherichiacoli',
 3: 'Escherichiacoli',
 4: 'Escherichiacoli',
 5: 'Escherichiacoli',
 6: 'Escherichiacoli',
 7: 'Escherichiacoli',
 8: 'Escherichiacoli',
 9: 'Escherichiacoli',
 10: 'Escherichiacoli',
 11: 'Escherichiacoli',
 12: 'Escherichiacoli',
 13: 'Escherichiacoli',
 14: 'Escherichiacoli',
 15: 'Escherichiacoli',
 16: 'Escherichiacoli',
 17: 'Escherichiacoli',
 18: 'Escherichiacoli',
 19: 'Escherichiacoli',
 20: 'Escherichiacoli',
 21: 'Escherichiacoli',
 22: 'Escherichiacoli',
 23: 'Escherichiacoli',
 24: 'Escherichiacoli',
 25: 'Escherichiacoli',
 26: 'Escherichiacoli',
 27: 'Escherichiacoli',
 28: 'Escherichiacoli',
 29: 'Escherichiacoli',
 30: 'Escherichiacoli',
 31: 'Escherichiacoli',
 32: 'Escherichiacoli',
 33: 'Escherichiacoli',
 34: 'Escherichiacoli',
 35: 'Escherichiacoli',
 36: 'Escherichiacoli',
 37: 'Escherichiacoli',
 38: 'Escherichiacoli',
 39: 'Escherichiacoli',
 40: 'Escherichiacoli',
 41: 'Escherichiacoli',
 4